<a href="https://colab.research.google.com/github/sanjanasajith/AI-Generated-Text-Detection/blob/main/AI_Generated_Text_Detection_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
from transformers import TrainingArguments, Trainer, AutoModelForSequenceClassification, AutoTokenizer
from datasets import Dataset
import pandas as pd
import torch
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Load data
data = pd.read_csv('/content/LLM.csv')
data.columns = ['text', 'label']

# Convert textual labels to numerical values
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['label'])

# Drop rows with NaN labels and ensure valid labels
data = data.dropna(subset=['label'])
data = data[data['label'].isin([0, 1])]

# Convert the DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(data)

# Split the dataset
train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

# Load tokenizer and model
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2).to("cuda")

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)

# Apply tokenization
train_dataset = train_dataset.map(tokenize_function, batched=True)
eval_dataset = eval_dataset.map(tokenize_function, batched=True)

# Remove unnecessary columns
train_dataset = train_dataset.remove_columns(["text"])
eval_dataset = eval_dataset.remove_columns(["text"])

# Training arguments
training_args = TrainingArguments(
    output_dir='/kaggle/working/',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='/kaggle/working/',
    logging_steps=10,
    report_to="none",
)

# Compute metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = (predictions == labels).mean()
    return {"accuracy": accuracy}

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Train the model on a small subset to estimate time
fraction = 0.1
train_subset = train_dataset.select(range(int(len(train_dataset) * fraction)))
start_time = time.time()
trainer.train_dataset = train_subset
trainer.train()
end_time = time.time()

# Estimate full training time
training_time_fraction = end_time - start_time
estimated_full_training_time = training_time_fraction / fraction
print(f"Estimated full training time: {estimated_full_training_time:.2f} seconds")

# Train on the full dataset
trainer.train_dataset = train_dataset
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

# Real-time input loop for predictions
print("\nEnter a sentence to classify (type 'exit' to quit):")
model.eval()

while True:
    user_input = input("Your sentence: ")
    if user_input.lower() == 'exit':
        break

    # Tokenize the user input
    inputs = tokenizer(user_input, truncation=True, padding=True, max_length=512, return_tensors="pt").to("cuda")

    # Make prediction
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        prediction = torch.argmax(logits, dim=-1).item()

    # Convert prediction to label
    predicted_label = label_encoder.inverse_transform([prediction])[0]
    print(f"Predicted label: {predicted_label}\n")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-10-9b2d29b8ad13>:68: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.394587,1.000000
2,0.495300,0.254620,1.000000
3,0.495300,0.218867,1.000000


Estimated full training time: 530.89 seconds


Epoch,Training Loss,Validation Loss,Accuracy
1,0.006700,0.004342,1.000000
2,0.002000,0.002612,1.000000
3,0.001600,0.002404,1.000000


Evaluation results: {'eval_loss': 0.002403580117970705, 'eval_accuracy': 1.0, 'eval_runtime': 6.2271, 'eval_samples_per_second': 35.49, 'eval_steps_per_second': 2.248, 'epoch': 3.0}

Enter a sentence to classify (type 'exit' to quit):
Predicted label: student

Your sentence: The error "ModuleNotFoundError: 
Predicted label: ai



In [ ]:
!pip install datasets



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.6 MB/s eta 0:00:00
